This is the sample solution to week 5 lab coding exercises. The lab uses two input data files: *ratings.csv* and *movies.csv*.

The *movies.csv* file contains movie information. Each row represents one movie, and has the following format:

`movieId,title,genres`

The *ratings.csv* file contains rating information. Each row represents one rating of one movie by one user, and has the following format:

`userId,movieId,rating,timestamp`


### Connect to Spark

In [17]:
from pyspark import SparkConf, SparkContext

spark_conf = SparkConf()\
        .setAppName("Week 5 Lecture Sample Code")
sc=SparkContext.getOrCreate(spark_conf) 


###  Q1. Find movies with no genre list


In [18]:
# As only MID is needed, we do not use the csv parsing feature,
# but simply split the row by comma and take the first item

movieData = sc.textFile("movies.csv")
movieData.filter(lambda row: '(no genres listed)' in row) \
    .map(lambda row: row.split(",")[0]) \
    .collect()

['126929', '135460', '138863', '141305', '141472', '143709', '149532']

### Q2, the top 5 movie of genre `Documentary'

In [3]:
# same rating extraction code as used in lecture
def extractRating(record):
    try:
        userID, movieID, rating, timestamp = record.split(",")
        rating = float(rating)
        return (movieID, rating)
    except:
        return ()

In [4]:
# create a list of movie ids in Documentary genre
# To enable join, which happens between two paird RDDs 
# we add an value 1 indicating it appears once
docMovie = movieData.filter(lambda row: 'Documentary' in row) \
    .map(lambda row: (row.split(",")[0],1))

# extract rating from the rating file
ratingData = sc.textFile("ratings.csv")
movieRatings=ratingData.map(extractRating)

# join the two
# then group based on ids, 
# and compute the lengh of value list
doc_rating = docMovie.join(movieRatings)
doc_rating.groupByKey() \
    .mapValues(len) \
    .sortBy(lambda r:r[1],ascending=False) \
    .take(5)

[('5669', 51), ('246', 36), ('2064', 35), ('8464', 33), ('8622', 33)]

In [5]:
# an alternative using reduceByKey

doc_rating.reduceByKey(lambda v1, v2: (v1[0]+v2[0],0)) \
    .mapValues(lambda v: v[0]) \
    .sortBy(lambda r:r[1],ascending=False) \
    .take(5)


[('5669', 51), ('246', 36), ('2064', 35), ('8464', 33), ('8622', 33)]

In [6]:
# another alternative using countByKey
mid_count = doc_rating.countByKey().items()
mid_count_rdd = sc.parallelize(mid_count)
mid_count_rdd.sortBy(lambda r:r[1],ascending=False) \
    .take(5)

[('5669', 51), ('246', 36), ('2064', 35), ('8464', 33), ('8622', 33)]

In [7]:
# inspect the element of joined RDD: (movie_id: (1, rating))
doc_rating.take(2)

[('108', (1, 2.0)), ('162', (1, 4.0))]

### Q3. Find most common co-occurring genres

In [19]:
import csv
"""
This module includes a few functions used in computing average rating per genre
"""
def getGenrePairs(record):
    """This function converts entries of movies.csv into ((g1,g2),1) pair for all genres 
    appearing in the row. 
    since there may be multiple genre per movie, this function returns a list of tuples
    Args:
        record (str): A row of CSV file, with three columns separated by comma
    Returns:
        The return value is a list of tuples, each tuple contains ((g1,g2), 1)
    """
    for row in csv.reader([record]):
        if len(row) != 3:
            return []
        genre_list = row[2].split("|")
        g = len(genre_list)
        if g<2 : #single genre case
            return []
        # at least two genre case
        results = []
        sorted_glist = sorted(genre_list) # sort by aphabet order

        for i in range(g): 
            for j in range(i+1,g): # from 1 to last
                results.append(((sorted_glist[i],sorted_glist[j]),1))
        return results

In [20]:
movieData.flatMap(getGenrePairs)\
    .reduceByKey(lambda a,b:a+b) \
    .sortBy(lambda r: r[1],ascending=False).take(5)


[(('Drama', 'Romance'), 1096),
 (('Comedy', 'Drama'), 1039),
 (('Drama', 'Thriller'), 1016),
 (('Comedy', 'Romance'), 892),
 (('Crime', 'Drama'), 841)]